In [1]:
import os
import psycopg
import pandas as pd
import re
from dotenv import load_dotenv
load_dotenv(dotenv_path=r"C:\Users\milom\Documents\landman\.env")

conn = psycopg.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
)

In [2]:
# Load a table into a pandas DataFrame
query = "SELECT * FROM clerk.documents_header;"
df = pd.read_sql(query, conn)

C:\Users\milom\AppData\Local\Temp\ipykernel_23348\1288731701.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
df

,id,doc_number,source_county,recorded_date,doc_type,book_vol_page,legal_description,doc_link,search_term,scraped_at,doc_path
0,1,1970-883,Freestone,1970-04-16,RELEASE OF LIEN,DR/396/743,AB#0142 E CLEMENTS SUR(54.2 ACRES),https://freestone.tx.publicsearch.us/doc/94458756,Alford John,2025-09-30 15:58:21.438617,None
1,2,1967-1470,Freestone,1967-06-13,WARRANTY DEED,DR/369/241,Survey- Survey: 0142 Acres: 54.2,https://freestone.tx.publicsearch.us/doc/13241...,Alford John,2025-09-30 15:58:21.438617,None
2,3,1983-2781,Freestone,1983-05-06,RELEASE,DR/636/894,CLEMENTS E A-0142,https://freestone.tx.publicsearch.us/doc/94539776,Alford John,2025-09-30 15:58:21.438617,None
3,4,1951-1884,Freestone,1951-08-22,ROYALTY DEED,DR/219/300,S SANCHEZ LEAGUE,https://freestone.tx.publicsearch.us/doc/26549...,Alford John,2025-09-30 15:58:21.438617,None
4,5,1951-2159,Freestone,1951-09-19,ROYALTY DEED,DR/219/577,JH EVANS JR SUR,https://freestone.tx.publicsearch.us/doc/26549...,Alford John,2025-09-30 15:58:21.438617,None
...,...,...,...,...,...,...,...,...,...,...,...
531,533,1956-1944,Freestone,1956-07-10,RATIFICATION,DR/270/17,WH STONE SUR,https://freestone.tx.publicsearch.us/doc/26550...,Emma Stone,2025-10-02 00:27:28.154697,None
532,534,1952-1041,Freestone,1952-03-25,RATIFICATION,DR/225/434,WH STONE SUR,https://freestone.tx.publicsearch.us/doc/26549...,Emma Stone,2025-10-02 00:27:28.154697,None
533,537,1952-1039,Freestone,1952-03-25,RATIFICATION,DR/225/424,WH STONE SUR,https://freestone.tx.publicsearch.us/doc/26549...,Emma Stone,2025-10-02 00:27:28.154697,None
534,538,2011-1103707,Freestone,2011-07-29,AGREEMENT,OR/1555/584,MULTIPLE TRACTS SEE INSTRUMENT,https://freestone.tx.publicsearch.us/doc/94746980,Emma Stone,2025-10-02 00:27:28.154697,None


In [4]:
def acres_string_to_float(acreage_str):
    """Convert a string like '1 1/2', '1/5', '40', or '.5' into a float."""
    if not acreage_str or not isinstance(acreage_str, str):
        return None

    # Remove all characters except digits, dot, slash, and space
    cleaned = re.sub(r'[^0-9./\s]', '', acreage_str).strip()

    # Mixed number (e.g. '1 1/2')
    if re.match(r'^\d+\s+\d+/\d+$', cleaned):
        whole, num, den = map(float, re.findall(r'\d+', cleaned))
        return round(whole + num / den, 6)

    # Pure fraction (e.g. '1/5')
    if re.match(r'^\d+/\d+$', cleaned):
        num, den = map(float, re.findall(r'\d+', cleaned))
        return round(num / den, 6)

    # Decimal (e.g. '.5', '5.44')
    if re.match(r'^\d*\.\d+$', cleaned):
        return round(float(cleaned), 6)

    # Whole number (e.g. '40')
    if re.match(r'^\d+$', cleaned):
        return round(float(cleaned), 6)

    # If none of the above matched, return None
    print('failed to convert string')
    return None

First clean legal description by splitting into:  Abstract number, survey name, acreage  (null if none), subdivision info, misc , case number (If doc type is abstract judgment)


In [5]:
def parse_legal_description(legal_desc, doc_type):
    """Extracts survey name and abs number from a single legal description string."""
    if not isinstance(legal_desc, str):
        return pd.Series([None, None, None, None, None, None])
    
    abs_patterns = [
        r'AB#\d+',
        r'A-\d+',
        r'Survey: \d+'
    ]
    survey_patterns = [
        r'.*LEAGUE',
        r'.*SUR',
        r'.*GRANT',
        r'Survey- Name: .*'
    
    ]
    unwanted_survey_phrases = [
        'Survey- Name: ',
        'Survey Name: ',
        'Survey-',
        'Survey -',
        'GRANT',
        'SURVEY',
        'SUR',
        'LEAGUE',


    ]


    acreage_patterns = [
        # --- Decimals (with parentheses) ---
        r'\(\d+\.\d+\s*ACRES\)',             # (5.44 ACRES), 5.44 ACS
        r'\(\.\d+\s*ACRES\)',                # (.5 ACRES), (.5 ACS)
        r'\(\d+\.\d+\s*ACS\)',             # (5.44 ACRES), 5.44 ACS
        r'\(\.\d+\s*ACS\)',                # (.5 ACRES), (.5 ACS)

        # --- Decimals (without parentheses) ---
        r'\d*\.\d+\s*ACRES',             # (5.44 ACRES), 5.44 ACS
        r'\.\d+\s*ACRES',                # (.5 ACRES), (.5 ACS)
        r'\d*\.\d+\s*ACS',             # (5.44 ACRES), 5.44 ACS
        r'\.\d+\s*ACS',                # (.5 ACRES), (.5 ACS)


        # --- Mixed numbers (whole + fraction) ---
        r'\(\d+\s+\d+/\d+\s*ACRES\)',        # (1 1/2 ACRES), 1 1/2 ACS
        r'\d+\s+\d+/\d+\s*ACRES',

        # --- Pure fractions ---
        r'\(\d+/\d+\s*ACRES\)',              # (1/5 ACRES), 1/5 ACS
        r'\d+/\d+\s*ACRES',

        # --- Whole numbers ---
        r'\(PT \d*\.?\d+ ACRES\)', # (PT 5.44 ACRES)
        r'\(\d+\s*ACRES\)', 
        r'\d+\s*ACRES',                  # (5 ACRES), 5 ACS
        r'\d+ ACS',

        # --- Labelled or unusual formatting ---
       r'Acres?:\s*\d*\.?\d+', # Acres: 54.2 Acres: .96
       r'Acres: \d'

        


    ]
    subdivision_patterns = [   
        r'L-\w+ (?:\b\w+\b\s+)*ADDN',   
        r'ADDN \d \d{5} \d{5} \d{3}[A-Z]',
        r'ADDN \d{5} \d{5} \d{3}[A-Z]',
        r'ADDN \d+',
        r'SUBD LOT \d+',
        r'SUBD \d+',
        r'\d{5} \d{4} \d{4}[A-Z] \d{4}',
        r'\d{5} \d{5} \d{4}',
        r'\d{5} \d{5} \d{3}[A-Z]',
        r'L-\w+ (?:\b\w+\b\s+)*',
        r'\w+ \d{5}',
        r'Subdivision\s?-',
    

        


    ]
    misc_legal_patterns = [
        r'MULTIPLE TRACTS SEE INSTRUMENT',
        r'N/A',

    ]


    
    abs_num = None
    survey_name = None
    acreage = None
    subdivision = None
    case_number = None
    misc_legal = None

    if doc_type == 'ABSTRACT JUDGEMT':
        case_number = legal_desc.strip()
        return pd.Series([abs_num, survey_name, acreage, subdivision, case_number, misc_legal])

    #Checks for undescriptive legal descriptions to return as misc_legal
    for pattern in misc_legal_patterns:
        match = re.search(pattern, legal_desc, re.IGNORECASE)
        if match:
            misc_legal = legal_desc.strip()
            return pd.Series([abs_num, survey_name, acreage, subdivision, case_number, misc_legal]) 

    #Extract Columns from legal description
    # ABS Number
    for pattern in abs_patterns:
        match = re.search(pattern, legal_desc)
        if match:
            abs_num = match.group(0)
            legal_desc = legal_desc.replace(abs_num, '').strip()
            # keep only digits from the match
            abs_num = re.sub(r'\D', '', abs_num)
            break
    
    #Numeric value of Acreage
    for pattern in acreage_patterns:
        match = re.search(pattern, legal_desc, re.IGNORECASE)
        if match:
            acreage_str=match.group(0)
            legal_desc=legal_desc.replace(acreage_str,'').strip()
            acreage=acres_string_to_float(acreage_str)
            break



    
    for pattern in subdivision_patterns:
        match = re.search(pattern, legal_desc, re.IGNORECASE)
        if match:
            subdivision = legal_desc 
            return pd.Series([abs_num, survey_name, acreage, subdivision, case_number, misc_legal])

    # Survey Name

    for pattern in survey_patterns:
        match = re.search(pattern, legal_desc)
        if match:
            survey_name = match.group(0)
            legal_desc = legal_desc.replace(survey_name, '').strip()
            break
        if not match:
            if abs_num or acreage:
                survey_name = legal_desc.strip()
                legal_desc = legal_desc.replace(survey_name, '').strip()
                break


    misc_legal = legal_desc.strip() 
    if survey_name:
        for phrase in unwanted_survey_phrases:
            if phrase in survey_name:
                survey_name = survey_name.replace(phrase, "")
                misc_legal += f" st: {phrase}"
        survey_name = survey_name.strip()

    if survey_name=='':
        survey_name = None

    

    if misc_legal=='':
        misc_legal = None
        
    
    

    
    return pd.Series([abs_num, survey_name, acreage, subdivision, case_number, misc_legal])

In [6]:
df[['abs_num', 'survey_name', 'acreage', 'subdivision', 'case_number', 'misc_legal']] = (
    df[['legal_description', 'doc_type']]
    .apply(lambda row: parse_legal_description(row['legal_description'], row['doc_type']), axis=1)
)



In [8]:
df[['doc_number', 'doc_type', 'legal_description', 'abs_num', 'survey_name', 'acreage', 'subdivision', 'case_number', 'misc_legal']].sample(50)


,doc_number,doc_type,legal_description,abs_num,survey_name,acreage,subdivision,case_number,misc_legal
368,1978-726,WARRANTY DEED,L-3A ISIAH CARDEN ADDN,None,None,NaN,L-3A ISIAH CARDEN ADDN,None,None
378,1995-3171,EXCHANGE DEED,J Y AGUILERA GRANT A-0002 1.389 ACS,0002,J Y AGUILERA,1.389,None,None,st: GRANT
82,1978-966,MECHANICS LIEN,AB#0014 WMC JONES LEAGUE(4 ACRES),0014,WMC JONES,4.000,None,None,st: LEAGUE
138,1979-2513,WARRANTY DEED,L-25 EASTVIEW ADDN,None,None,NaN,L-25 EASTVIEW ADDN,None,None
27,1976-4828,DEED OF TRUST,L-2 B-4 FAIRFIELD TOWN OF,None,None,NaN,L-2 B-4 FAIRFIELD TOWN OF,None,None
489,1977-730,MECHANICS LIEN,D CARPENTER SUR(2 ACRES),None,D CARPENTER,2.000,None,None,st: SUR
92,1978-757,DEED OF TRUST,AB#0012 R GAINER LEAGUE(10.000 ACRES),0012,R GAINER,10.000,None,None,st: LEAGUE
122,1975-3845,MECHANICS LIEN,L-4 B-20 FAIRFIELD CITY OF,None,None,NaN,L-4 B-20 FAIRFIELD CITY OF,None,None
341,2011-1103861,GENERAL W/D,J MC WHORTER SUR A-0428 16.202 ACS,0428,J MC WHORTER,16.202,None,None,st: SUR
323,1986-877,WARNTY DEED,SHEPHERD H A-0034,0034,SHEPHERD H,NaN,None,None,None
